In [1]:
import requests
import time

In [2]:
import os

folder_path = "sample_data"
text_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

documents = {}
for file_name in text_files:
    with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
        documents[file_name] = file.read()
all_text = "\n\n".join(documents.values())
documents

{'Data Catalogs.txt': 'In einem modernen Unternehmen, in dem Daten aus unterschiedlichsten Quellen – etwa ERP-Systemen, IoT-Plattformen, Webtracking-Tools oder externen Datenlieferanten – zusammenfließen, wird es zunehmend entscheidend, nicht nur die Daten selbst, sondern auch deren Kontext, Herkunft und Nutzungsmöglichkeiten zu dokumentieren, was durch systematische Metadatenpflege und den Einsatz von Data Catalogs ermöglicht wird; ein gut gepflegter Datenkatalog wirkt wie eine Landkarte im Dickicht komplexer Datenlandschaften: Er zeigt, welche Daten wo liegen, wie sie aufgebaut sind, wer sie nutzt, wofür sie gebraucht werden und wie vertrauenswürdig sie sind – und ermöglicht damit nicht nur eine bessere Auffindbarkeit, sondern auch eine koordinierte Wiederverwendung, Governance und Qualitätssicherung, was insbesondere in datengetriebenen Transformationsprojekten enorme Effizienzvorteile bringt.',
 'Data Governance.txt': 'Wenn Unternehmen wachsen und sich ihre Prozesse, Systeme und Ab

In [ ]:
def query_mistral(model, prompt):
    url = "http://ollama:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    start_time = time.time()
    response = requests.post(url, json=payload)
    end_time = time.time()

    result = response.json()["response"]
    duration = end_time - start_time

    print(f"Response: {result}")
    print(f"\n⏱️ Time taken: {duration:.2f} seconds")

query_mistral("mistral", "Kannst du mir auf Deutsch antworten?")
#query_mistral("llama2", "Kannst du mir auf Deutsch antworten?")

In [13]:
question = "What is the main topic covered across the five documents?"
full_prompt = f"""The following is a collection of text documents:

{all_text}

Based on this, answer the following question: {question}
"""
query_mistral("mistral", full_prompt)

Response:  The main topic covered across the five documents is Data Management, specifically focusing on data governance, data quality, data strategy, role models in data management, and the overall importance of these aspects for modern businesses.

⏱️ Time taken: 131.94 seconds
